In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [2]:
BH_tweets_during_war = pd.read_csv('dane/BH_tweets_during_war.csv', lineterminator='\n')

In [3]:
BH_tweets_during_war.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,5.583900e+04,5.583900e+04,55839.0,5.583900e+04,55839.000000,55839.000000,55839.000000,55839.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.445762e+18,1.445355e+18,200.0,4.892263e+17,0.700192,1.438027,8.473468,0.352102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,4.180217e+16,4.344011e+16,0.0,5.838775e+17,10.028606,37.626933,178.207916,0.477630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.377411e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.404538e+18,1.404424e+18,200.0,1.850421e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.447847e+18,1.447604e+18,200.0,2.794160e+09,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.486599e+18,1.486477e+18,200.0,1.112632e+18,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.509678e+18,1.509678e+18,200.0,1.508394e+18,1088.000000,6621.000000,32103.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_during_war.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55839 entries, 0 to 55838
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               55839 non-null  int64  
 1   conversation_id  55839 non-null  int64  
 2   created_at       55839 non-null  object 
 3   date             55839 non-null  object 
 4   time             55839 non-null  object 
 5   timezone         55839 non-null  int64  
 6   user_id          55839 non-null  int64  
 7   username         55839 non-null  object 
 8   name             55839 non-null  object 
 9   place            841 non-null    object 
 10  tweet            55839 non-null  object 
 11  language         55839 non-null  object 
 12  mentions         55839 non-null  object 
 13  urls             55839 non-null  object 
 14  photos           55839 non-null  object 
 15  replies_count    55839 non-null  int64  
 16  retweets_count   55839 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_during_war.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1509677705280868360,1509677686700089348,2022-04-01 01:43:08 CEST,2022-04-01,01:43:08,200,1093447620893331457,peregreine,Peregrine Trader,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1509675515128471555,1509675515128471555,2022-04-01 01:34:25 CEST,2022-04-01,01:34:25,200,162060435,traceyryniec,Tracey Ryniec,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1509671718545235969,1509671718545235969,2022-04-01 01:19:20 CEST,2022-04-01,01:19:20,200,332364947,simonsalhanyre,BHHS Simon & Salhany,"{'type': 'Point', 'coordinates': [41.0379043, ...",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1509670638801334272,1509670638801334272,2022-04-01 01:15:03 CEST,2022-04-01,01:15:03,200,2738590284,kevinhillre,Kevin Hill,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1509667550791094281,1509667550791094281,2022-04-01 01:02:47 CEST,2022-04-01,01:02:47,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format and sorted by dates
BH_tweets_during_war['date'] = pd.to_datetime(BH_tweets_during_war['date'], format = '%Y-%m-%d')
BH_tweets_during_war = BH_tweets_during_war.sort_values(by='date')
BH_tweets_during_war = BH_tweets_during_war.reset_index(drop=True)
BH_tweets_during_war

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1377410706518142978,1377410706518142978,2021-04-01 02:01:18 CEST,2021-04-01,02:01:18,200,1130525203619037185,11graphs,11Graphs,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1377662110239498240,1377662110239498240,2021-04-01 18:40:17 CEST,2021-04-01,18:40:17,200,49660367,realtordebby_tn,"Debby Hill, REALTOR",NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1377656478228693003,1377304839949066240,2021-04-01 18:17:54 CEST,2021-04-01,18:17:54,200,878743468981583874,angelapetree5,Angela Petree,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'MileyCyrus', 'name': 'Miley ...",NaN,NaN,NaN,NaN
3,1377653739868323840,1377609588292386829,2021-04-01 18:07:01 CEST,2021-04-01,18:07:01,200,143526150,z3itg3ist,~ Geppetto - LivinG VirtuaL ID is BORN not MADE,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Jfperrau', 'name': 'J.Franco...",NaN,NaN,NaN,NaN
4,1377653298484903941,1377653298484903941,2021-04-01 18:05:16 CEST,2021-04-01,18:05:16,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55834,1509654314792464395,1509654314792464395,2022-04-01 00:10:11 CEST,2022-04-01,00:10:11,200,732736306569170944,francie_grieb,Francie grieb,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
55835,1509652247260635136,1509652247260635136,2022-04-01 00:01:58 CEST,2022-04-01,00:01:58,200,2344133624,jana_realtor,Jana Xu,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
55836,1509651973217341445,1509651973217341445,2022-04-01 00:00:53 CEST,2022-04-01,00:00:53,200,316914016,ilvre,LasVegasRealEstate,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
55837,1509663472807727106,1509663472807727106,2022-04-01 00:46:34 CEST,2022-04-01,00:46:34,200,47431775,lhamptonteam,Louise Hampton Team,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [7]:
#Remove tweets writeen in other language than english
BH_tweets_during_war_clean = BH_tweets_during_war[BH_tweets_during_war.language == 'en']
BH_tweets_during_war_clean = BH_tweets_during_war_clean.reset_index(drop=True)
BH_tweets_during_war_clean.shape

(49808, 36)

In [8]:
#Remove columns without any data
BH_tweets_during_war_clean = BH_tweets_during_war_clean.dropna(thresh = 1, axis = 1)

In [9]:
#3782 records and 10 columns were removed
BH_tweets_during_war_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,4.980800e+04,4.980800e+04,49808.0,4.980800e+04,49808.000000,49808.000000,49808.000000,49808.000000
mean,1.446015e+18,1.445609e+18,200.0,4.833736e+17,0.699265,1.445089,8.246667,0.366246
std,4.156761e+16,4.326079e+16,0.0,5.819440e+17,10.558428,39.726622,187.554921,0.481783
min,1.377411e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000
25%,1.406159e+18,1.405845e+18,200.0,1.776624e+08,0.000000,0.000000,0.000000,0.000000
50%,1.448260e+18,1.447983e+18,200.0,2.768300e+09,0.000000,0.000000,0.000000,0.000000
75%,1.486373e+18,1.486302e+18,200.0,1.095783e+18,0.000000,0.000000,1.000000,1.000000
max,1.509678e+18,1.509678e+18,200.0,1.508394e+18,1088.000000,6621.000000,32103.000000,1.000000


In [10]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49808 entries, 0 to 49807
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               49808 non-null  int64         
 1   conversation_id  49808 non-null  int64         
 2   created_at       49808 non-null  object        
 3   date             49808 non-null  datetime64[ns]
 4   time             49808 non-null  object        
 5   timezone         49808 non-null  int64         
 6   user_id          49808 non-null  int64         
 7   username         49808 non-null  object        
 8   name             49808 non-null  object        
 9   place            840 non-null    object        
 10  tweet            49808 non-null  object        
 11  language         49808 non-null  object        
 12  mentions         49808 non-null  object        
 13  urls             49808 non-null  object        
 14  photos           49808 non-null  objec

In [11]:
BH_tweets_during_war_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1377410706518142978,1377410706518142978,2021-04-01 02:01:18 CEST,2021-04-01,02:01:18,200,1130525203619037185,11graphs,11Graphs,NaN,...,0,0,"['financials', 'bank', 'insurance']","['xlf', 'brk.b', 'jpm', 'bac', 'wfc', 'c']",https://twitter.com/11Graphs/status/1377410706...,False,NaN,0,NaN,[]
1,1377662110239498240,1377662110239498240,2021-04-01 18:40:17 CEST,2021-04-01,18:40:17,200,49660367,realtordebby_tn,"Debby Hill, REALTOR",NaN,...,0,0,[],[],https://twitter.com/RealtorDebby_TN/status/137...,False,NaN,1,https://pbs.twimg.com/media/Ex5wYulWYAYJMQO.jpg,[]
2,1377656478228693003,1377304839949066240,2021-04-01 18:17:54 CEST,2021-04-01,18:17:54,200,878743468981583874,angelapetree5,Angela Petree,NaN,...,0,0,[],[],https://twitter.com/AngelaPetree5/status/13776...,False,NaN,0,NaN,"[{'screen_name': 'MileyCyrus', 'name': 'Miley ..."
3,1377653298484903941,1377653298484903941,2021-04-01 18:05:16 CEST,2021-04-01,18:05:16,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,0,0,['homesforsale'],[],https://twitter.com/diamondRAfargo/status/1377...,False,NaN,1,https://pbs.twimg.com/media/Ex5oX50WgAE3m5b.jpg,[]
4,1377653292260589573,1377653292260589573,2021-04-01 18:05:15 CEST,2021-04-01,18:05:15,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,0,0,['realestate'],[],https://twitter.com/diamondRAfargo/status/1377...,False,NaN,1,https://pbs.twimg.com/media/Ex5oXj4W8AUQkAa.jpg,[]


In [12]:
#Check values of column place
BH_tweets_during_war_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [44.6981658, -93.2177749]}      134
{'type': 'Point', 'coordinates': [28.16028, -82.38634]}           50
{'type': 'Point', 'coordinates': [33.7864851, -84.38743]}         33
{'type': 'Point', 'coordinates': [33.82291021, -116.5135278]}     25
{'type': 'Point', 'coordinates': [40.78806845, -74.47095514]}     17
                                                                ... 
{'type': 'Point', 'coordinates': [38.47389, -121.41686]}           1
{'type': 'Point', 'coordinates': [39.37909026, -76.65567061]}      1
{'type': 'Point', 'coordinates': [40.8279741, -81.1245418]}        1
{'type': 'Point', 'coordinates': [33.78589024, -84.38724273]}      1
{'type': 'Point', 'coordinates': [41.0684459, -81.5082323]}        1
Name: place, Length: 198, dtype: int64

In [13]:
#Check values of column quote_url
BH_tweets_during_war_clean['quote_url'].value_counts()

https://twitter.com/CashApp/status/1390698245970616320           9
https://twitter.com/mytradesignals/status/1383293035652345858    8
https://twitter.com/RBReich/status/1435348653284237318           7
https://twitter.com/bitpanda/status/1485923043209879552          5
https://twitter.com/CNN/status/1410293330366320648               5
                                                                ..
https://twitter.com/SamRo/status/1489248348553027584             1
https://twitter.com/CaltransDist7/status/1440684313624731656     1
https://twitter.com/TheBuffaloNews/status/1488701204058230787    1
https://twitter.com/SlowGoncalves/status/1406186888302563333     1
https://twitter.com/AaronPyne4/status/1504917404320772102        1
Name: quote_url, Length: 1171, dtype: int64

In [14]:
#Counted quantity of links for each tweet
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_during_war_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].astype(np.int64)

In [15]:
#Replayce NaN to [] in thumbnail column
BH_tweets_during_war_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [16]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49808 entries, 0 to 49807
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               49808 non-null  int64         
 1   conversation_id  49808 non-null  int64         
 2   created_at       49808 non-null  object        
 3   date             49808 non-null  datetime64[ns]
 4   time             49808 non-null  object        
 5   timezone         49808 non-null  int64         
 6   user_id          49808 non-null  int64         
 7   username         49808 non-null  object        
 8   name             49808 non-null  object        
 9   place            840 non-null    object        
 10  tweet            49808 non-null  object        
 11  language         49808 non-null  object        
 12  mentions         49808 non-null  object        
 13  urls             49808 non-null  object        
 14  photos           49808 non-null  objec

In [17]:
#Check duplicates
BH_tweets_during_war_clean[BH_tweets_during_war_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [18]:
#Column retweet consist of only False values
BH_tweets_during_war_clean['retweet'].value_counts()

False    49808
Name: retweet, dtype: int64

In [19]:
#Removed irrelevant columns
BH_tweets_during_war_clean = BH_tweets_during_war_clean.drop(['conversation_id','created_at', 'time', 
                                                              'timezone', 'language', 'retweet', 'link','place'], 
                                                             axis = 1)

In [20]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_during_war_clean[column] == '[]'
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str[1:-1].str.split(',')
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str.len()
    BH_tweets_during_war_clean.loc[empty_list, column] = 0

In [21]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].astype(np.int64)

In [22]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_during_war_clean.shape[0]):
    BH_tweets_during_war_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_during_war_clean.loc[i, 'tweet']).split())

In [23]:
#Last day 2022-04-01 has only 14 tweets, it will be removed
BH_tweets_during_war_clean['date'].value_counts()

2021-05-03    1128
2021-05-01     958
2022-03-21     812
2021-05-02     784
2022-02-26     646
              ... 
2021-10-02      55
2021-04-18      54
2021-04-11      51
2021-11-28      39
2022-04-01      14
Name: date, Length: 366, dtype: int64

In [24]:
BH_tweets_during_war_clean = BH_tweets_during_war_clean[BH_tweets_during_war_clean['date'] != '2022-04-01']
BH_tweets_during_war_clean['date'].value_counts()

2021-05-03    1128
2021-05-01     958
2022-03-21     812
2021-05-02     784
2022-02-26     646
              ... 
2021-09-05      56
2021-10-02      55
2021-04-18      54
2021-04-11      51
2021-11-28      39
Name: date, Length: 365, dtype: int64

In [25]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_during_war_clean[~BH_tweets_during_war_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [26]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49794 entries, 0 to 49793
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              49794 non-null  int64         
 1   date            49794 non-null  datetime64[ns]
 2   user_id         49794 non-null  int64         
 3   username        49794 non-null  object        
 4   name            49794 non-null  object        
 5   tweet           49794 non-null  object        
 6   mentions        49794 non-null  int64         
 7   urls            49794 non-null  int64         
 8   photos          49794 non-null  int64         
 9   replies_count   49794 non-null  int64         
 10  retweets_count  49794 non-null  int64         
 11  likes_count     49794 non-null  int64         
 12  hashtags        49794 non-null  int64         
 13  cashtags        49794 non-null  int64         
 14  quote_url       49794 non-null  int64         
 15  vi

In [27]:
#Save cleaned data in csv format
BH_tweets_during_war_clean.to_csv('dane/BH_tweets_during_war_clean.csv', index=False)

In [28]:
#Create dataframe for textanalysis for each day
BH_tweets_during_war_textanalysis = pd.date_range(BH_tweets_during_war_clean['date'].min(),
                                                       BH_tweets_during_war_clean['date'].max(),freq='d')
BH_tweets_during_war_textanalysis = pd.DataFrame(BH_tweets_during_war_textanalysis, columns = ['date'])
column_names = ['Short negative factual tweets', 'Short negative personal tweets', 'Short positive factual tweets',
                'Short positive personal tweets', 'Short neutral factual tweets', 'Short neutral personal tweets',
                'Long negative factual tweets', 'Long negative personal tweets', 'Long positive factual tweets',
                'Long positive personal tweets', 'Long neutral factual tweets', 'Long neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_during_war_textanalysis[column] = 0
BH_tweets_during_war_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2021-04-01,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-04-02,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-04-03,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-04-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-04-05,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-03-27,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2022-03-28,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2022-03-29,0,0,0,0,0,0,0,0,0,0,0,0,0
363,2022-03-30,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_during_war_clean.shape[0]):
    blob = TextBlob(BH_tweets_during_war_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if len(blob.words) < 30:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral personal tweets'] += 1
    else:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral personal tweets'] += 1
    BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [30]:
BH_tweets_during_war_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2021-04-01,1,0,14,21,34,0,3,4,3,18,7,0,105
1,2021-04-02,4,2,14,8,26,0,3,2,10,9,5,0,83
2,2021-04-03,2,0,8,6,18,0,2,1,12,6,6,1,62
3,2021-04-04,0,0,1,16,21,1,2,1,7,14,4,0,67
4,2021-04-05,1,1,25,19,52,0,5,0,11,7,5,1,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-03-27,4,3,14,59,25,1,4,3,20,14,3,1,151
361,2022-03-28,0,2,21,31,37,3,3,1,19,24,6,0,147
362,2022-03-29,3,2,19,23,40,1,3,0,23,22,3,2,141
363,2022-03-30,4,4,33,22,33,0,4,1,17,16,8,0,142


In [31]:
BH_tweets_during_war_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            365 non-null    datetime64[ns]
 1   Short negative factual tweets   365 non-null    int64         
 2   Short negative personal tweets  365 non-null    int64         
 3   Short positive factual tweets   365 non-null    int64         
 4   Short positive personal tweets  365 non-null    int64         
 5   Short neutral factual tweets    365 non-null    int64         
 6   Short neutral personal tweets   365 non-null    int64         
 7   Long negative factual tweets    365 non-null    int64         
 8   Long negative personal tweets   365 non-null    int64         
 9   Long positive factual tweets    365 non-null    int64         
 10  Long positive personal tweets   365 non-null    int64         
 11  Long n

In [32]:
#Save text analysis data in csv format
BH_tweets_during_war_textanalysis.to_csv('dane/BH_tweets_during_war_textanalysis.csv', index=False)